# <strong>Bitcoin Recurrent Neural Network<strong>
### Justin Marlor & Habit Blunk
##### *Colorado State University*

In [1]:
# Imports
import pandas as pd
import subprocess

result = subprocess.run(['bash', './add-run-kaggle-bitcoin'], capture_output=True,text=True)

print(result.stdout)
print(result.stderr)

if result.returncode == 0:
  df = pd.read_csv("kaggle-bitcoin/upload/btcusd_1-min_data.csv")
  display(df)

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: kaggle-bitcoin (0.1.0)

The lock file might not be compatible with the current version of Poetry.
Upgrade Poetry to ensure the lock file is read properly or, alternatively, regenerate the lock file with the `poetry lock` command.
Traceback (most recent call last):
  File "/home/Jmarlor/CS345/bitcoin-rnn/kaggle-bitcoin/kaggle_bitcoin/kaggle_update_bitcoin.py", line 5, in <module>
    import kaggle
  File "/home/Jmarlor/.cache/pypoetry/virtualenvs/kaggle-bitcoin-BCoBIEpq-py3.12/lib/python3.12/site-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/home/Jmarlor/.cache/pypoetry/virtualenvs/kaggle-bitcoin-BCoBIEpq-py3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/Jmarlor/.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import GRU